# Создание процессов

Создание процесса на Python
* Как создать дочерний процесс?
* Как работает системный вызов fork?
* Модуль multiprocessing


## Пример №1

Процесс в системе создается при помощи системного вызова `fork`.  
`fork` создает точную копию родительского процесса, вся память, все файловые дескрипторы, все ресурсы которые были доступны в родительском процессе будут целиком и полностью скопированы в дочерний процесс. Как только отработал системный вызов `fork`, с этого момента у нас 2 процесса:

In [ ]:
import time
import os

p = os.fork() # Это системный вызов который точную копию родительского процесса

if p == 0:
    # Этот код исполняется в дочернием процессе
    while True:
        print('child pid: {}, for break press <CTRL+C>'.format(os.getpid()))
        time.sleep(5)
else:
    # Этот код исполняется в родительском процессе
    print('parent pid: {}'.format(os.getpid()))
    os.wait() # Это системный вызов, он позволяет дожидаться завершения выполнения дочернего процесса


parent pid: 4263
child pid: 4264, for break press <CTRL+C>
child pid: 4264, for break press <CTRL+C>
child pid: 4264, for break press <CTRL+C>
child pid: 4264, for break press <CTRL+C>


Поизучаем наши процессы командой `ps`. Здесь мы видим 2 процесса с PID 67502 и 67503:

In [21]:
%%bash

ps aux | head -1 && ps aux | grep process1.py

USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
ubuntu     4263  0.0  0.1  29812  7972 pts/0    S+   11:43   0:00 python3 process1.py
ubuntu     4264  0.0  0.0  29812  5440 pts/0    S+   11:43   0:00 python3 process1.py
ubuntu     4269  0.0  0.0  14792   804 ?        S    11:43   0:00 grep process1.py


А с помощью флага `-f` команды `ps` можно посмотреть процессы в иерархическом представлении:

In [22]:
%%bash

ps auf | head -1 && ps auf | grep process1.py

USER        PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
ubuntu     4263  0.0  0.1  29812  7972 pts/0    S+   11:43   0:00                  \_ python3 process1.py
ubuntu     4264  0.0  0.0  29812  5440 pts/0    S+   11:43   0:00                      \_ python3 process1.py


Посмотрим, что делают процессы? Для этого воспользуемся командой `strace`.  
Здесь мы видим, что дочерний процесс вызывает системный вызов `write()` и выводит информацию в стандартный поток вывода (STDOUT):

In [ ]:
%%bash

sudo strace -p 4227

strace: Process 4264 attached
select(0, NULL, NULL, NULL, {tv_sec=4, tv_usec=992259}) = 0 (Timeout)
getpid()                                = 4264
write(1, "child pid: 4264, for break press"..., 42) = 42
select(0, NULL, NULL, NULL, {tv_sec=5, tv_usec=0}) = 0 (Timeout)
getpid()                                = 4264
write(1, "child pid: 4264, for break press"..., 42) = 42


Выведем информацию о родительском процессе. Здесь процесс вызвал системный вызов `wait()` и дожидается, когда операционная система оповестит его о том, что дочерний процесс завершился:

In [ ]:
%%bash

sudo strace -p 4128

strace: Process 4263 attached
wait4(-1, 


## Пример №2

In [1]:
import os

foo = 'bar'

if os.fork() == 0: # Вызов системного вызова fork() который целиком копирует дочерний процесс.
    # Дочерний процесс.
    # Память полностью _скопирована_ в этот дочерний процесс и переменная foo объявленная
    # раньше точно также здесь доступна:
    print('child pid: {}, foo = {}'.format(os.getpid(), foo))
    foo = 'baz!' # Изменение значения переменной foo никак не повлияет на родительский процесс и её переменную foo
    print('child pid: {}, foo = {}'.format(os.getpid(), foo))
else:
    # Родительский процесс.
    # Переменная foo осталась с прежним значением, дочерний процесс на нее не влияет
    print('parent pid: {}, foo = {}'.format(os.getpid(), foo))
    os.wait() # Ожидание завершения работы дочернего процесса

child pid: 4512, foo = bar
child pid: 4512, foo = baz!
parent pid: 4499, foo = bar


## Пример №3

In [2]:
!cat file.txt

Line one
Line two
Line three


In [3]:
import os

f = open('file.txt')
foo = f.readline() # Считывание первой строки из файла

if os.fork() == 0:
    # Дочерний процесс
    foo = f.readline().strip() # Считывание второй строки из файла. Это никак не влияет на родительский процесс
    print('child pid: {}, foo = "{}"'.format(os.getpid(), foo))
else:
    # Родительский процесс
    foo = f.readline().strip() # Считывание второй строки из файла
    print('parent pid: {}, foo = "{}"'.format(os.getpid(), foo))

parent pid: 4499, foo = "Line two"
child pid: 4528, foo = "Line two"


Результат. Оба процесса считали вторую строку из файла.

## Пример №4

Создание процесса, модуль `multiprocessing`.  
Системный вызов `fork()` может вызвать ошибку, это нужно проверять. Обычно в Python используют модуль `multiprocessing` для создания процессов, который к тому же кросплатформенный, не на всех системах есть системный вызов `fork()`:

In [4]:
from multiprocessing import Process

def f(name):
    print('hello', name)

# Передаем функцию, которую хотим исполнить в дочернем процессе и аргументы этой функции:
p = Process(target=f, args=('Bob',))

# После того, как создан объект Process, никакого процесса в ОС создано не будет.
# Он будет создан только после того, как мы вызовем метод start():
p.start() # Этот метод вызывает системный вызов fork() в котором будет исполнена функция f
p.join()  # Важно дожидаться выполнение всех дочерних процессов, для этого используется метод join()

hello Bob


Также, существует альтернативный метод создания процесса при помощи наследования:

In [5]:
class PrintProcess(Process):

    def __init__(self, name):
        super().__init__()
        self.name = name

    def run(self):
        print('hello', self.name)


p = PrintProcess('Mike')
p.start()
p.join()

hello Mike


## Итоги

Мы рассмотрели:
* Как создать дочерний процесс?
* Работа системного вызова fork
* Модуль multiprocessing

Мы не говорили об:
* Обмене данными между процессами
* Синхронизации процессов